In [ ]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import hvplot.pandas

# Machine Learning
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the data into a Pandas DataFrame
df = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df.head(10)

In [ ]:
# Generate summary statistics
df.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
# initialize
scaler = StandardScaler()

# fit
scaler.fit(df)

# predict/transform
scaled_data = scaler.transform(df)

In [ ]:
# Create a DataFrame with the scaled data
# Copy the crypto names from the original data
df_scaled = pd.DataFrame(scaled_data, columns=df.columns)

# Set the coinid column as index
df_scaled.set_index(df.index, inplace=True)

# Display sample data
df_scaled.head(10) 

---

### Find the Best Value for k Using the Scaled Data.

#### Elbow Curve A: Inertia Only, Range 1 to 11

In [ ]:
# Define your "X" - features to predict
X = df_scaled

In [ ]:
# Create an empty list to store the inertia values
inertia_a = []

# Create a list with the number of k-values from 1 to 11
k_a = list(range(1, 11))

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for i in k_a:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    k_a_model = KMeans(n_clusters=i, random_state=1)
    
    # 2. Fit the model to the data using `df_scaled`
    k_a_model.fit(X)
    
    # predict the model
    preds_a = k_a_model.predict(X)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia_a.append(k_a_model.inertia_)

    
    print(f"Finished {i} out of {max(k_a)}")

In [ ]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data_a = {"k": k_a, "inertia": inertia_a}
df_elbow_a = pd.DataFrame(elbow_data_a)

df_elbow_a["acc"] = df_elbow_a.inertia.diff()

# Review the DataFrame
df_elbow_a.head(20)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# using matplotlib
plt.plot(df_elbow_a["k"], df_elbow_a["inertia"])
plt.title("Elbow Curve A")
plt.xticks(df_elbow_a["k"])
plt.ylabel("inertia")
plt.xlabel("k")
plt.show()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# using hv plot
elbow_plot = df_elbow_a.hvplot.line(
    width=600,
    height=400,
    x="k", 
    y="inertia", 
    title="Elbow Curve Using Scaled Data", 
    xticks="k")
elbow_plot

### Elbow Curve B: Inertia, Silhouette & Calinski Harabasz, Range 2 to 20 

In [ ]:
# Define "X" - features to predict
X = df_scaled

In [ ]:
#Booth's way
# Create an empty list to store the inertia values
inertia = []
silhouettes = []
cha_chas = []

# Create a list with the number of k-values from 2 to 20
k = list(range(2, 20))

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for i in k:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    k_model = KMeans(n_clusters=i, random_state=1)
    
    # 2. Fit the model to the data using `df_scaled`
    k_model.fit(X)
    
    # predict the model
    preds = k_model.predict(X)
    
    # 3. Append the inertia, silhouete and cha scores to their lists
    inertia.append(k_model.inertia_)
    score = silhouette_score(X, preds)
    silhouettes.append(score)
    
    cha_cha = calinski_harabasz_score(X, preds)
    cha_chas.append(cha_cha)
    
    print(f"Finished {i} out of {max(k)}")

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia, "silhouette_score": silhouettes, "cha_score": cha_chas}
df_elbow = pd.DataFrame(elbow_data)

df_elbow["acc"] = df_elbow.inertia.diff()

# Review the DataFrame
df_elbow.head(20)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
plt.plot(df_elbow["k"], df_elbow["inertia"])
plt.title("Elbow Curve B")
plt.xticks(df_elbow["k"])
plt.ylabel("inertia")
plt.xlabel("k")
plt.show()

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Scaled Data

In [ ]:
df_sub = df_scaled.copy()

# Define the model with the higher value of k clusters
# Use a random_state of 1 to generate the model

# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

# Fit the K-Means model using the scaled data
model.fit(X)

# Predict the clusters to group the cryptocurrencies using the scaled data
preds = model.predict(X)

# Print the resulting array of cluster values.
print(preds)

In [ ]:
# Create a copy of the DataFrame
df_sub2 = df_sub.copy()

# Add a new column to the DataFrame with the predicted clusters
df_sub2['clusters'] = preds

# Display sample data
df_sub2.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_plot = df_sub2.hvplot.scatter(
    width=600,
    height=400,
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    c="clusters",
    colormap='rainbow', #colormap, chat gbt
    hover_cols=['coin_id']).opts(bgcolor='darkgray') # background, chat gbt
scatter_plot

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
n_components = 3
pca=PCA(n_components=n_components)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_data = pca.fit_transform(df_scaled)

print(pca_data)

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance_ratio = pca.explained_variance_ratio_

print(explained_variance_ratio)

In [ ]:
sum(explained_variance_ratio[0:])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.8950316570309842

In [ ]:
# Create a new DataFrame with the PCA data.
df_pca = pd.DataFrame(pca_data, columns=["PC" + str(x) for x in range(1, n_components + 1)])

# Copy the crypto names from the original data
names = df_scaled.index

# Set the coinid column as index
df_pca.set_index(names, inplace=True)

# Display sample data
df_pca.head()

---

### Find the Best Value for k Using the PCA Data

#### Elbow Curve A: Inertia Only, Range 1 to 11

In [ ]:
# Define "X" - features to predict
X = df_pca

In [ ]:
# Create a list with the number of k-values from 1 to 11
inertia_a = []

# Create a list with the number of k-values from 1 to 11
k_a = list(range(1, 11))

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for i in k_a:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    k_a_model = KMeans(n_clusters=i, random_state=1)
    
    # 2. Fit the model to the data using `df_pca`
    k_a_model.fit(X)
    
    # predict the model
    preds_a = k_a_model.predict(X)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia_a.append(k_a_model.inertia_)

    print(f"Finished {i} out of {max(k_a)}")

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data_a = {"k": k_a, "inertia": inertia_a}
pca_df_elbow_a = pd.DataFrame(pca_elbow_data_a)

pca_df_elbow_a["acc"] = pca_df_elbow_a.inertia.diff()

# Review the DataFrame
pca_df_elbow_a.head(20)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# using matplotlib
plt.plot(pca_df_elbow_a["k"], pca_df_elbow_a["inertia"])
plt.title("Elbow Curve A (PCA)")
plt.xticks(pca_df_elbow_a["k"])
plt.ylabel("inertia")
plt.xlabel("k")
plt.show()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# using hv plot
elbow_plot_pca = pca_df_elbow_a.hvplot.line(
    width=600,
    height=400,
    x="k", 
    y="inertia", 
    title="Elbow Curve Using PCA Data", 
    xticks="k")
elbow_plot_pca

### Elbow Curve B: Inertia, Silhouette & Calinski Harabasz, Range 2 to 20 

In [ ]:
#define X 
X = df_pca

In [ ]:
#Booth's way
# Create an empty list to store the inertia values
inertia = []
silhouettes = []
cha_chas = []

# Create a list with the number of k-values from 2 to 20
k = list(range(2, 20))

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for i in k:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    k_model = KMeans(n_clusters=i, random_state=1)
    
    # 2. Fit the model to the data using `df_pca`
    k_model.fit(X)
    
    # predict the model
    preds = k_model.predict(X)
    
    # 3. Append the inertia, silhouete and cha scores to their lists
    inertia.append(k_model.inertia_)
    score = silhouette_score(X, preds)
    silhouettes.append(score)
    cha_cha = calinski_harabasz_score(X, preds)
    cha_chas.append(cha_cha)
    
    print(f"Finished {i} out of {max(k)}")

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
# Define a DataFrame to hold the values for k and the corresponding inertia
pca_elbow_data = {"k": k, "inertia": inertia, "silhouette_score": silhouettes, "cha_score": cha_chas}
pca_df_elbow = pd.DataFrame(pca_elbow_data)

pca_df_elbow["acc"] = pca_df_elbow.inertia.diff()

# Review the DataFrame
pca_df_elbow.head(20)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
# using matplot lib
plt.plot(pca_df_elbow["k"], pca_df_elbow["inertia"])
plt.title("Elbow Curve B (PCA)")
plt.xticks(pca_df_elbow["k"])
plt.ylabel("inertia")
plt.xlabel("k")
plt.show()

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

# Fit the K-Means model using the PCA data
model.fit(X)

# Predict the clusters to group the cryptocurrencies using the PCA data
preds = model.predict(X)

# Print the resulting array of cluster values.
print(preds)

In [ ]:
# Create a copy of the DataFrame with the PCA data
df_pca_sub = df_pca.copy()

# Predict the clusters to group the cryptocurrencies using the PCA data
df_pca_sub['clusters'] = preds

# Display sample data
df_pca_sub.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_plot_pca = df_pca_sub.hvplot.scatter(
    width=600,
    height=400,
    x="PC1",
    y="PC2",
    c="clusters",
    colormap='rainbow', #colormap, chat gbt
    hover_cols=['coin_id']).opts(bgcolor='darkgray') #background, chat gbt
scatter_plot_pca

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
composite_elbow = elbow_plot + elbow_plot_pca
composite_elbow

In [ ]:
# Composite plot to contrast the clusters
composite_scatter = scatter_plot + scatter_plot_pca
composite_scatter

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features (through PCA) does not significantly affect the performance of the K-Means clustering in this case. The clustering patterns are preserved, and the elbow curve indicates that the optimal number of clusters remains consistent.